## PRÁCTICA FINAL BLOCKCHAIN
### Alejandro Manuel López Gómez

La forma de utilizar la aplicación es la siguiente.
- 1º Arrancar el script "Web Application" "Node 8000" y "Node 8001" y conectarse mediante interfaz web a la dirección dada por el script "Web Application"
- 2º Conectarse a uno de los nodos disponibles, cuya dirección habrá sido dada por los scripts
- 3º Una vez dentro del interfaz web de la red blockchain, el primer paso es registrar un usuario junto con un valor de módulo. Para ello se debe emplear el código "Generate Key Pair", con el que se generarán el par de claves y la firma de transacciones.
- 4º Introducir en el formulario de registro de usuario el nombre deseado y el valor del módulo, en el momento en el que sean introducidos esta asociación se guardará para validar futuras transacciones.
- 5º A continuación se deben introducir transacciones, para ello se debe emplear el código "Generate Key Pair" de nuevo. EL NOMBRE DE USUARIO INTRODUCIDO DEBE CONCORDAR CON EL NOMBRE REGISTRADO EN EL PASO ANTERIOR. El usuario introducirá el nombre del autor de la transacción (su propio usuario), el conteido y la firma generada por el código "Generate Key Pair".

In [ ]:
import matplotlib.pyplot as plt
import hashlib
import time
import json
from flask import Flask, request
import json
import requests
import time

from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import binascii

class Block():
    def __init__(self,index,transactions,timestamp,previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        
    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys = True)
        return hashlib.sha256(block_string.encode('utf-8')).hexdigest()

class Blockchain():
    def __init__(self,difficulty):
        self.chain = []
        self.difficulty = difficulty
        self.unconfirmed_transactions = []
        self.create_genesis_block()
        
    def create_genesis_block(self):
        genesis_block = Block(0,[],time.time(),"0")
        genesis_block.current_hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
    
    def last_block(self):
        return self.chain[-1]
    
    def genesis_block(self):
        return self.chain[0]
    
    def proof_of_work(self,block):
        block.nonce = 0
        while True:
            block_hash = block.compute_hash()
            if (block_hash[:self.difficulty] == "".join(str(i) for i in [0]*self.difficulty)):
                break
            block.nonce += 1
        return block_hash
    
    def is_valid_proof(self,block,block_hash):
        if block_hash[:self.difficulty] == "".join(str(i) for i in [0]*self.difficulty) and block_hash == block.compute_hash():
            return True
        return False
    
    def append_block(self,block,block_hash):
        if block.previous_hash == self.last_block().current_hash and self.is_valid_proof(block,block_hash):
            block.current_hash = block_hash
            self.chain.append(block)
            return True
        return False

    def add_new_transaction(self,transaction):
        self.unconfirmed_transactions.append(transaction)
    
    def mine(self):
        if self.unconfirmed_transactions:
            new_block = Block(self.last_block().index+1,self.unconfirmed_transactions,time.time(),self.last_block().current_hash)
            proof = self.proof_of_work(new_block)
            self.append_block(new_block,proof)
            self.unconfirmed_transactions = []
            return self.last_block().index
        return False
    
    def check_chain(self,chain):
        for b in range(2,len(chain)):
            current_hash = chain[b].current_hash
            delattr(chain[b],'current_hash')
            if not (self.is_valid_proof(chain[b],current_hash) and chain[b].previous_hash == chain[b-1].current_hash):
                return False
            chain[b].current_hash = current_hash
        return True
    
app = Flask(__name__)
blockchain = Blockchain(2)
peers = set()
public_keys = {}
e = 0x10001
srv_port = "8000"

@app.route('/register_new_user',methods = {'POST'})
def new_user():
    data = request.get_json()
    required_fields = ['user','public_key']

    if not all(f in data for f in required_fields):
        return ("Invalid user registration data", 404)
    
    try:
        if data['user'] not in public_keys:
            public_keys[data['user']] = int(data['public_key'])
            data = {"user" : data['user'], "public_key" : data['public_key']}
            headers = {'Content-Type' : "application/json"}

            if peers:
                for node in peers:
                    response = requests.post(node+"/update_key_list",json=data,headers=headers)
                    if response.status_code != 200:
                        return ("Error while registering user, a node failed to register", 400)

            return ("User registration successful",201)
        return ("Error while registering user", 400)
    
    except ValueError:
        return ("Public key must be a number",400)
    
@app.route('/update_key_list',methods = {'POST'})
def update_key_list():
    data = request.get_json()

    if data['user'] not in public_keys:
        public_keys[data['user']] = int(data['public_key'])
        return ("User registered",200)
    return ("User already exists",400)
    
@app.route('/new_transaction',methods = {'POST'})
def new_transaction():
    data = request.get_json()
    required_fields = ["author","content","signature"]

    if not all(f in data for f in required_fields):
        return ("Invalid transaction data", 404)
    
    if verify_signature(data):
        data['timestamp'] = time.time()
        blockchain.add_new_transaction(data)
        return ("Success", 201)
    return ("Signature verification failed",400)

@app.route('/chain',methods = {'GET'})
def get_chain():
    data = {"length": len(blockchain.chain),
            "chain" : [b.__dict__ for b in blockchain.chain],
            "peers" : list(peers),
            "public_keys" : public_keys}
    return (json.dumps(data), 200)

@app.route('/mine',methods = {'GET'})
def mine():
    consensus()
    backup_unconfirmed_transactions = blockchain.unconfirmed_transactions 
    if blockchain.unconfirmed_transactions:
        new_index = blockchain.mine()
    
        if not consensus():
            announce_new_block(blockchain.last_block(),request)
            if peers:
                data = "Block has been mined and announced. New_block_index : " +str(new_index)
                return (data, 201)
            return ("Block was mined for current node, but not announced (no registered peers)", 201)
    
    blockchain.unconfirmed_transactions = backup_unconfirmed_transactions
    return ("Block was discarded or no pending transactions to mine", 400)

@app.route('/pending_transactions', methods = {'GET'})
def get_pending_transactions():
    if blockchain.unconfirmed_transactions:
        data = {"unconfirmed_transactions" : blockchain.unconfirmed_transactions}
        return (json.dumps(data), 200)
    return ("No unconfirmed transactions", 404)

@app.route('/register_new_node', methods = {'POST'})
def register_new_node():
    new_node_address = request.get_json()['new_node_address']
    if not new_node_address:
        return ("Invalid new node data", 400)
    
    if new_node_address not in peers:
        if new_node_address[:7] != "http://":
            peers.add("http://"+new_node_address)
        else:
            peers.add(new_node_address)
            
    return get_chain()

@app.route('/register_with_existing_node', methods = {'POST'})
def register_with_existing_node():
    node_address = request.get_json()['node_address']
    if not node_address:
        return ("Invalid node data", 400)
    data = {"new_node_address" : request.host_url[:-1]}
    headers = {'Content-Type' : "application/json"}
    
    response = requests.post("http://"+node_address+"/register_new_node",json=data,headers=headers)
    
    if response.status_code == 200:
        chain_dump = response.json()['chain']
        peer_dump = response.json()['peers']
        key_dump = response.json()['public_keys']
        
        new_blockchain = Blockchain(2)
        new_blockchain.chain = []
        genesis_block = Block(chain_dump[0]['index'],chain_dump[0]['transactions'],chain_dump[0]['timestamp'],chain_dump[0]['previous_hash'])
        genesis_block.current_hash = genesis_block.compute_hash()
        new_blockchain.chain.append(genesis_block)
        
        for b in range(1,len(chain_dump)):
            block = Block(chain_dump[b]['index'],chain_dump[b]['transactions'],chain_dump[b]['timestamp'],chain_dump[b]['previous_hash'])
            block_hash = new_blockchain.proof_of_work(block)
            if not new_blockchain.append_block(block,block_hash) or not verify_signature(chain_dump[b]['transactions']):
                print("Invalid block")
        
        blockchain.chain = new_blockchain.chain
        peers.add("http://"+node_address)
        
        for p in peer_dump:
            if p!= request.host_url[:-1]:
                peers.add(p)
        
        for k in key_dump.keys():
            if k not in public_keys.keys():
                public_keys[k] = key_dump[k]
        
        return ("Registration successful",200)
    else:
        return response.content, response.status_code

@app.route('/add_block', methods = {'POST'})
def add_block():
    data = request.get_json()
    required_fields = ["index","transactions","timestamp","previous_hash","current_hash"]
    
    if not all(f in data for f in required_fields):
        return ("Invalid transaction data", 404)
    
    block = Block(data['index'],data['transactions'],data['timestamp'],data['previous_hash'])
    block_hash = data['current_hash']
    blockchain.proof_of_work(block)

    if not blockchain.append_block(block,block_hash) or not verify_signature(data['transactions']):
        return ("Block has been discarded",400)
    return ("Success",200)

def announce_new_block(block, request):
    for peer in peers:
        if peer != request.host_url:
            data = json.dumps(block.__dict__,sort_keys = True)
            headers = {"Content-Type" : "application/json"}
            response = requests.post(peer+"/add_block",data=data,headers=headers)
            
def consensus():
    global blockchain
    longest_chain = None
    current_len = len(blockchain.chain)
    
    for node in peers:
        current_chain = (requests.get(node+"/chain")).json()["chain"]
        if len(current_chain) > current_len:
            current_len = len(current_chain)
            longest_chain = chain
            
    if longest_chain:
        blockchain.chain = longest_chain
        return True
    return False

def verify_signature(transaction_data):
    if isinstance(transaction_data,list):
        for transaction in transaction_data:
            author = transaction['author']
            content = transaction['content']
            signature = int(transaction['signature'])

            transaction_json = {"author" : author, "content" : content}
            transaction_hash = int(hashlib.sha256(json.dumps(transaction_json,sort_keys=True).encode('utf8')).hexdigest(),16)

            if author in public_keys:
                n = int(public_keys[author])
                decrypted_signature = pow(signature,e,n)
                if decrypted_signature != transaction_hash:
                    return False
        return True
    else:
        author = transaction_data['author']
        content = transaction_data['content']
        signature = int(transaction_data['signature'])

        transaction_json = {"author" : author, "content" : content}
        transaction_hash = int(hashlib.sha256(json.dumps(transaction_json,sort_keys=True).encode('utf8')).hexdigest(),16)

        if author in public_keys:
            n = int(public_keys[author])
            decrypted_signature = pow(signature,e,n)
            if decrypted_signature == transaction_hash:
                    return True
        return False

if __name__ == "__main__":
    app.run(port=srv_port)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/May/2023 10:30:23] "POST /register_new_user HTTP/1.1" 201 -
127.0.0.1 - - [13/May/2023 10:30:27] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2023 10:30:58] "POST /new_transaction HTTP/1.1" 201 -
127.0.0.1 - - [13/May/2023 10:31:02] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2023 10:31:03] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2023 10:31:06] "GET /pending_transactions HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2023 10:31:08] "GET /mine HTTP/1.1" 201 -
127.0.0.1 - - [13/May/2023 10:31:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/May/2023 10:31:12] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2023 10:31:13] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2023 10:31:15] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2023 10:31:32] "POST /new_transaction HTTP/1.1" 400 -
127.0.0.1 - - [13/May/2023 10:31:33] "GET /pending_transactions HTTP/1.1" 404 -
127.0.0.1 - - [13/May/202